# What are initialization bias correction terms (Sec 3 in the paper)
(a.k.a How did you get $\hat{m_t}$ and $\hat{v_t}$)

## Problem Statement
------------------------------------------------------------
Let $g$ be the gradient of the stochastic objective $f$,
and $g_1, ..., g_T$ be the gradients at subsequent timesteps, each a draw from an
underlying gradient distribution $g_t \sim p(g_t)$. 
Let the initial moving average $v_0 = 0$ (a vector of zeros).
Estimate its second
raw moment (uncentered variance) using an exponential moving average of the squared gradient,
with decay rate $\beta_2$. 

----------------------------------------------------------------

## Answer
In order to find bias-corrected estimator for $g_t \odot g_t$, we want to know how $E[v_t]$ is different from $E[g_t^2]$. 
First observe that if we expand the recursive relations of $v_t,$ $v_t = \beta_2 v_{t-1} + (1 - \beta_2) (g_t \odot g_t)$ can be re-written as
a function of previous gradients:
$$
v_t = \beta_2^t v_0 + (1-\beta_2) \sum_{i=1}^t \beta_2^{t-i} \cdot g_i \odot g_i \\
= (1-\beta_2) \sum_{i=1}^t \beta_2^{t-i} \cdot g_i \odot g_i
$$
since $v_0 = 0$. Recall that $g_t$ is a random variable, drawn from some probability distribution of gradients. Since $v_t$ is a function of random variable $g_i$s, $v_t$ is a random variable too. Taking the expectation of $v_t$ yields
$$
E[v_t] = (1 - \beta_2) E[\sum_{i=1}^t \beta_2^{t-i} \cdot g_i \odot g_i]  \\
= (1 - \beta_2) \sum_{i=1}^t \beta_2^{t-i} \cdot E[g_i \odot g_i]   \quad \text{(Linearity of Expectation)} 
$$
Denote $g_i \odot g_i$ as $g_i^2$ for convenience. Then,
$$
E[v_t] = (1 - \beta_2) \sum_{i=1}^t \beta_2^{t-i} \cdot E[g_i^2] \\
= (1 - \beta_2) \left ( \beta_2^0 E[g_t^2] + \beta_2^1 E[g_{t-1}^2] + \beta_2^2 E[g_{t-2}^2] + \cdots + \beta_2^{t-1} E[g_1^2] \right ) \\
= 1 \cdot E[g_t^2] + \beta_2^1 (E[g_{t-1}^2] - E[g_t^2]) + \beta_2^2 (E[g_{t-2}^2]  - E[g_{t-1}^2]) + \cdots + \beta_2^{t-1} (E[g_1^2] - E[g_2^2]) - \beta_2^{t} E[g_1^2]   \\
\approx ( 1 - \beta_2^t) \cdot E[g_t^2]
$$
Therefore, the algorithm divides $v_t$ by this term $(1 - \beta_2^t)$ to get a bias-corrected version. 

Note that $\beta_2^t$ will be decreased exponentially as $t$ becomes large. This means that dividing $v_t$ by $1 - \beta_2^t$ will have an effect only in the initial epochs, where $t$ is small. 

The "memory" of the algorithm is controlled by $\beta_2$. Intuitively, the average number of epochs that the algorithm remembers updates is approximately $\frac{1}{1 - \beta_2}$. For example, with the default value of $\beta_2 = 0.999$, the algorithm will remember the information of the first update up to epoch 1000 (= 1 / (1 - 0.999)). 

## Intuitions behind Adam's update rules

### Adam's update rule 
$$
m_0 = v_0 = 0 \\
m_t =  \beta_1 m_{t-1} + (1- \beta_1) g_t \\
v_t = \beta_2 v_{t-1} + (1 - \beta_2) (g_t \odot g_t) \\
\hat{m_t} = m_t (bias-corrected)
$$

where $\odot$ is the element-wise product. 

35min + 1:30min + 40min (reading the paper included)

Note:
http://exochronos.hatenablog.com/entry/2016/05/07/194033